In [1]:
#prepare low-quality sample list and delete them from training dataset
#对低质量的样本进行剔除

import pandas as pd

#利用预处理阶段得到的低质量样本清单
sample_list_low_quality=['train/cat/cat.7564.jpg','train/dog/dog.11299.jpg','train/dog/dog.1194.jpg','train/cat/cat.4338.jpg','train/dog/dog.4367.jpg','train/dog/dog.8898.jpg','train/dog/dog.3889.jpg','train/cat/cat.5418.jpg','train/dog/dog.9188.jpg','train/cat/cat.7377.jpg','train/dog/dog.1773.jpg','train/dog/dog.1308.jpg','train/cat/cat.5351.jpg','train/dog/dog.10801.jpg','train/cat/cat.11184.jpg','train/cat/cat.7968.jpg','train/dog/dog.2614.jpg','train/dog/dog.9517.jpg','train/cat/cat.10029.jpg','train/dog/dog.6475.jpg','train/cat/cat.2433.jpg','train/dog/dog.6725.jpg','train/cat/cat.12493.jpg','train/dog/dog.8736.jpg','train/cat/cat.9171.jpg','train/dog/dog.12376.jpg','train/cat/cat.8456.jpg','train/dog/dog.10747.jpg','train/dog/dog.5604.jpg','train/dog/dog.10161.jpg','train/dog/dog.10237.jpg','train/dog/dog.1895.jpg','train/cat/cat.3672.jpg','train/dog/dog.11731.jpg','train/dog/dog.4334.jpg','train/dog/dog.7413.jpg','train/cat/cat.1407.jpg','train/cat/cat.6699.jpg','train/cat/cat.7575.jpg','train/cat/cat.2040.jpg','train/cat/cat.12424.jpg','train/cat/cat.4321.jpg','train/cat/cat.9596.jpg','train/cat/cat.9444.jpg','train/cat/cat.9589.jpg','train/cat/cat.5534.jpg','train/cat/cat.5324.jpg','train/cat/cat.9331.jpg','train/cat/cat.10712.jpg','train/cat/cat.8138.jpg','train/cat/cat.7362.jpg','train/cat/cat.3399.jpg','train/cat/cat.8504.jpg','train/cat/cat.7574.jpg','train/cat/cat.9639.jpg','train/cat/cat.92.jpg','train/cat/cat.6987.jpg','train/cat/cat.12272.jpg','train/cat/cat.9954.jpg','train/cat/cat.4985.jpg','train/cat/cat.5836.jpg','train/cat/cat.252.jpg','train/cat/cat.10536.jpg','train/cat/cat.3216.jpg','train/cat/cat.11222.jpg','train/cat/cat.7604.jpg','train/cat/cat.7920.jpg','train/cat/cat.10893.jpg','train/cat/cat.8828.jpg','train/cat/cat.11724.jpg','train/cat/cat.241.jpg','train/cat/cat.4085.jpg','train/cat/cat.6402.jpg','train/cat/cat.8100.jpg','train/cat/cat.5583.jpg','train/cat/cat.6215.jpg','train/cat/cat.9513.jpg','train/cat/cat.4688.jpg','train/cat/cat.9882.jpg','train/cat/cat.2939.jpg','train/cat/cat.7703.jpg','train/cat/cat.10266.jpg','train/cat/cat.12499.jpg','train/cat/cat.372.jpg','train/cat/cat.6515.jpg','train/cat/cat.10863.jpg','train/cat/cat.10775.jpg','train/cat/cat.8044.jpg','train/cat/cat.5527.jpg','train/cat/cat.44.jpg','train/cat/cat.5355.jpg','train/cat/cat.10151.jpg','train/cat/cat.6.jpg','train/cat/cat.8280.jpg','train/cat/cat.10181.jpg','train/cat/cat.5820.jpg','train/cat/cat.11870.jpg','train/cat/cat.3731.jpg','train/cat/cat.9783.jpg','train/cat/cat.6868.jpg','train/cat/cat.11565.jpg','train/cat/cat.8012.jpg','train/cat/cat.7194.jpg','train/cat/cat.1450.jpg','train/cat/cat.9250.jpg','train/cat/cat.335.jpg','train/cat/cat.10539.jpg','train/cat/cat.724.jpg','train/cat/cat.10107.jpg','train/cat/cat.7487.jpg']

#Load training dataset
#加载训练图片数据集
from sklearn.datasets import load_files     
data = load_files('train')
import numpy as np
from keras.utils import np_utils
train_files = np.array(data['filenames'])
train_targets = np_utils.to_categorical(np.array(data['target']), 2)   

#exclude low quality samples from the dataset
#从训练集中剔除低质量样本
train_files_final=[]
train_targets_final=[]
for train_file,train_target in zip(train_files,train_targets):
    if train_file not in sample_list_low_quality:
        train_files_final.append(train_file)
        train_targets_final.append(train_target)
print('final train file number: '+str(len(train_targets_final)))

Using TensorFlow backend.


final train file number: 24890


In [2]:
#function to extract features given image path
from keras.preprocessing import image   
import numpy as np
from keras.applications.nasnet import NASNetLarge, preprocess_input

#convert image to input data for model
def getdatafromimage(img_path):
    img = image.load_img(img_path, target_size=(331, 331))
    x = image.img_to_array(img)
    tensor=np.expand_dims(x, axis=0)
    imagedata=preprocess_input(tensor) #normalize image data using tf's preprocess_input to [-1,1]
    return imagedata

#extract features
pretrained_model=NASNetLarge(include_top=False, weights='imagenet',pooling='max',input_shape = (331, 331, 3))

def extract_features(train_file):
    imagedata=getdatafromimage(train_file)
    features=pretrained_model.predict(imagedata)
    return features.tolist()[0]

In [3]:
#extract features for each images in the final training set
from ipywidgets import IntProgress
from IPython.display import display
import time

p = IntProgress()
progressFull=len(train_files_final)
display(p)

train_features_final=[]
for train_file in train_files_final:
    features=extract_features(train_file)
    train_features_final.append(features)
    p.value = float(len(train_features_final))/float(progressFull)*100
    p.description = str(len(train_features_final))+'/'+str(progressFull)

IntProgress(value=0)

In [4]:
#store extracted features into csv files
import pandas as pd
train_features_final_df=pd.DataFrame(train_features_final)
train_features_final_df.to_csv('train_features_final.csv')
train_targets_final_df=pd.DataFrame(train_targets_final)
train_targets_final_df.to_csv('train_targets_final.csv')


In [5]:
#extract features for testing images
from sklearn.datasets import load_files     
data = load_files('test')
import numpy as np
from keras.utils import np_utils
test_files = np.array(data['filenames'])

from ipywidgets import IntProgress
from IPython.display import display
import time

p = IntProgress()
progressFull=len(test_files)
display(p)

test_features=[]
for test_file in test_files:
    features=extract_features(test_file)
    test_features.append(features)
    p.value = float(len(test_features))/float(progressFull)*100
    p.description = str(len(test_features))+'/'+str(progressFull)

IntProgress(value=0)

In [6]:
#store extracted features into csv files
import pandas as pd
test_features_df=pd.DataFrame(test_features)
test_features_df.to_csv('test_features.csv')